## 准备数据

In [97]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, optimizers, datasets

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # or any {'0', '1', '2'}

def mnist_dataset():
    (x, y), (x_test, y_test) = datasets.mnist.load_data()
    #normalize
    x = x/255.0
    x_test = x_test/255.0
    
    return (x, y), (x_test, y_test)

In [98]:
print(list(zip([1, 2, 3, 4], ['a', 'b', 'c', 'd'])))
(x, y), (x_test, y_test) = datasets.mnist.load_data()
print(x.shape)

[(1, 'a'), (2, 'b'), (3, 'c'), (4, 'd')]
(60000, 28, 28)


## 建立模型

In [99]:
class myModel:
    def __init__(self):
        ####################
        '''声明模型对应的参数'''
        ####################
        self.W1 = tf.Variable(shape=[28*28, 150], dtype=tf.float32,initial_value=tf.random.uniform(shape=[28*28, 150],minval=-0.1, maxval=0.1))
        self.b1 = tf.Variable(shape=[150], dtype=tf.float32, initial_value=tf.zeros(150))
        self.W2 = tf.Variable(shape=[150, 15], dtype=tf.float32,initial_value=tf.random.uniform(shape=[150, 15],minval=-0.1, maxval=0.1))
        self.b2 = tf.Variable(shape=[15], dtype=tf.float32, initial_value=tf.zeros(15))
    def __call__(self, x):
        ####################
        '''实现模型函数体，返回未归一化的logits'''
        ####################
        x_flat = tf.reshape(x, shape=[-1, 28*28])
        H_layer = tf.tanh(tf.matmul(x_flat, self.W1) + self.b1)
        logits = tf.matmul(H_layer, self.W2) + self.b2
        return logits
        
model = myModel()

optimizer = optimizers.Adam()

## 计算 loss

In [100]:
@tf.function
def compute_loss(logits, labels):
    return tf.reduce_mean(
        tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logits, labels=labels))

@tf.function
def compute_accuracy(logits, labels):
    predictions = tf.argmax(logits, axis=1)
    return tf.reduce_mean(tf.cast(tf.equal(predictions, labels), tf.float32))

@tf.function
def train_one_step(model, optimizer, x, y):
    with tf.GradientTape() as tape:
        logits = model(x)
        loss = compute_loss(logits, y)

    # compute gradient
    trainable_vars = [model.W1, model.W2, model.b1, model.b2]
    grads = tape.gradient(loss, trainable_vars)
    for g, v in zip(grads, trainable_vars):
        v.assign_sub(0.01*g)

    accuracy = compute_accuracy(logits, y)

    # loss and accuracy is scalar tensor
    return loss, accuracy

@tf.function
def test(model, x, y):
    logits = model(x)
    loss = compute_loss(logits, y)
    accuracy = compute_accuracy(logits, y)
    return loss, accuracy

## 实际训练

In [101]:
train_data, test_data = mnist_dataset()
for epoch in range(50):
    loss, accuracy = train_one_step(model, optimizer, 
                                    tf.constant(train_data[0], dtype=tf.float32), 
                                    tf.constant(train_data[1], dtype=tf.int64))
    print('epoch', epoch, ': loss', loss.numpy(), '; accuracy', accuracy.numpy())
loss, accuracy = test(model, 
                      tf.constant(test_data[0], dtype=tf.float32), 
                      tf.constant(test_data[1], dtype=tf.int64))

print('test loss', loss.numpy(), '; accuracy', accuracy.numpy())

epoch 0 : loss 2.6772566 ; accuracy 0.061433334
epoch 1 : loss 2.6623104 ; accuracy 0.069816664
epoch 2 : loss 2.6476452 ; accuracy 0.07875
epoch 3 : loss 2.6332512 ; accuracy 0.087516665
epoch 4 : loss 2.6191182 ; accuracy 0.09685
epoch 5 : loss 2.6052365 ; accuracy 0.106233336
epoch 6 : loss 2.5915968 ; accuracy 0.116
epoch 7 : loss 2.5781915 ; accuracy 0.12628333
epoch 8 : loss 2.5650113 ; accuracy 0.13708334
epoch 9 : loss 2.5520487 ; accuracy 0.14783333
epoch 10 : loss 2.5392966 ; accuracy 0.16025
epoch 11 : loss 2.526748 ; accuracy 0.17293334
epoch 12 : loss 2.514396 ; accuracy 0.18495
epoch 13 : loss 2.5022337 ; accuracy 0.1974
epoch 14 : loss 2.490255 ; accuracy 0.20955
epoch 15 : loss 2.4784553 ; accuracy 0.22141667
epoch 16 : loss 2.4668276 ; accuracy 0.23281667
epoch 17 : loss 2.4553668 ; accuracy 0.24513334
epoch 18 : loss 2.444068 ; accuracy 0.25675
epoch 19 : loss 2.432926 ; accuracy 0.26855
epoch 20 : loss 2.421936 ; accuracy 0.28011668
epoch 21 : loss 2.4110935 ; accura